In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Define paths

In [2]:
train_dir = 'split_data/train'
val_dir = 'split_data/val'
test_dir = 'split_data/test'

# Parameters

In [3]:
img_height, img_width = 128, 128
batch_size = 32
seed = 42

# Create ImageDataGenerator instances

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Training data generator

In [5]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    seed=seed
)

Found 1862 images belonging to 9 classes.


# Validation data generator

In [6]:
validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    seed=seed
)

Found 399 images belonging to 9 classes.


# Test data generator

In [7]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    seed=seed
)

Found 399 images belonging to 9 classes.


### Base model (ResNet50)

train from scratch


In [8]:
base_model = tf.keras.applications.Xception(include_top=False,
    weights=None,
    pooling='avg')

In [9]:
for layer in base_model.layers:
    layer.trainable = False

In [10]:
class_indices = train_generator.class_indices
class_names = list(class_indices.keys())
output =  tf.keras.layers.Dense(len(class_names), activation='softmax')(base_model.output)
new_model = tf.keras.Model(base_model.input, output)

# Define early stopping

In [11]:
es_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    verbose=1,
    patience=2,
    mode='max',
    restore_best_weights=True
)

# Compile the model

# Compile the model

In [12]:
new_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False,
    label_smoothing=0,
    reduction="auto",
    name="categorical_crossentropy"), metrics=['accuracy'])

# Train the model

In [13]:
history = new_model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=[es_callback]
)


Epoch 1/10
59/59 [==============================] - 163s 3s/step - loss: 2.1974 - accuracy: 0.0988 - val_loss: 2.1971 - val_accuracy: 0.1153
Epoch 2/10
59/59 [==============================] - 146s 2s/step - loss: 2.1972 - accuracy: 0.1139 - val_loss: 2.1970 - val_accuracy: 0.1128
Epoch 3/10
59/59 [==============================] - 150s 3s/step - loss: 2.1972 - accuracy: 0.1149 - val_loss: 2.1969 - val_accuracy: 0.1128
Epoch 3: early stopping


# Evaluate the model on the test set

In [14]:
test_loss, test_accuracy = new_model.evaluate(test_generator)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

13/13 [==============================] - 28s 2s/step - loss: 2.1971 - accuracy: 0.1128
Test Loss: 2.1971383094787598, Test Accuracy: 0.11278195679187775


In [15]:
image = 'prediction/g.jpeg'
image = tf.keras.utils.load_img(image, target_size=(img_height,img_width))
img_arr = tf.keras.utils.array_to_img(image)
img_bat=tf.expand_dims(img_arr,0)

In [16]:
predict = new_model.predict(img_bat)

1/1 [==============================] - 2s 2s/step


In [17]:
score = tf.nn.softmax(predict)

In [18]:
class_indices = train_generator.class_indices
class_names = list(class_indices.keys())

In [19]:
class_names

['Duryodhan',
 'Ganga',
 'Lali',
 'Parashuram',
 'Raju',
 'Sivilbahadur',
 'Sonu',
 'Tirathram',
 'Yoglaxmi']

In [20]:
print('Elephant in image is {} with accuracy of {:0.2f}'.format(class_names[np.argmax(score)],np.max(score)*100))

Elephant in image is Raju with accuracy of 11.29


In [21]:
new_model.save('alexnet_elephant_model.h5')